In [1]:
path = "/Users/astrachan/Desktop/outbrain/"
#which confidence level to eliminate
confidence_cut_category = 0.4
confidence_cut_topic = 0.2

#which portion of top scores to take
score_cut_category = 3
score_cut_topic = 3

In [2]:
import numpy as np
import pandas as p
import pickle

path_t = path + "source_tables/"
path_b = path_t + "built/"

document_on_ad_document_ctr = p.read_csv(path_b + "document_on_ad_document_ctr.csv", dtype={"document_id": int, "ad_document_id":int, "score": float})
documents_categories = p.read_csv(path_t + "documents_categories.csv", dtype = {"document_id":int, "category_id":int, "confidence_level":float})
documents_topics = p.read_csv("/Users/astrachan/Desktop/outbrain/source_tables/documents_topics.csv", dtype = {"document_id":int, "topic_id":int, "confidence_level":float})

In [4]:
#get only highest confidence topic and category
max_rows = documents_categories.groupby(['document_id'])['confidence_level'].transform(max) == documents_categories['confidence_level']
documents_categories = documents_categories[max_rows].drop_duplicates(subset = ['document_id','confidence_level'],keep = 'last')
max_rows = documents_topics.groupby(['document_id'])['confidence_level'].transform(max) == documents_topics['confidence_level']
documents_topics = documents_topics[max_rows].drop_duplicates(subset = ['document_id','confidence_level'],keep = 'last')
del max_rows

In [5]:
#remove category ids below a certain confidence level and merge promoted with documents_categories_reduced
documents_categories_reduced = documents_categories[documents_categories['confidence_level'] > confidence_cut_category].drop("confidence_level", axis = 1)
print 'category ids precent: ' + repr(float(len(documents_categories_reduced.category_id.unique())) / len(documents_categories.category_id.unique()))
print 'unique category ids: ' + repr(len(documents_categories_reduced.category_id.unique()))

category ids precent: 1.0
unique category ids: 95


In [6]:
#take only highest scored ad_on_doc
document_on_ad_document_ctr_categories = document_on_ad_document_ctr[:document_on_ad_document_ctr.shape[0] / score_cut_category]
print document_on_ad_document_ctr_categories.shape
print 'precentage taken: ' + repr(float(document_on_ad_document_ctr_categories.shape[0]) / document_on_ad_document_ctr.shape[0])
print 'minimal score taken: ' + repr(document_on_ad_document_ctr_categories.score.min())

(7719955, 3)
precentage taken: 0.3333333333333333
minimal score taken: 1.0352447964700002


In [7]:
print documents_categories_reduced.head()
print document_on_ad_document_ctr_categories.head()

   document_id  category_id
0      1595802         1611
2      1524246         1807
4      1617787         1807
6      1615583         1305
8      1615460         1613
   document_id  ad_document_id     score
0      1026091          898071  7.518608
1       961361         1290377  7.247171
2       961313         1585630  7.202495
3      2115556         1418830  7.192435
4      1976713          973567  7.044363


In [8]:
#merge with category ids of doc and ad
document_on_ad_document_ctr_categories = document_on_ad_document_ctr_categories.merge(documents_categories_reduced, how='left', on='document_id')
documents_categories_reduced.rename(columns={'document_id': 'ad_document_id'}, inplace=True)
document_on_ad_document_ctr_categories = document_on_ad_document_ctr_categories.merge(documents_categories_reduced, how='left', on='ad_document_id', suffixes=('_doc','_ad'))

In [9]:
document_on_ad_document_ctr_categories.isnull().sum()

document_id              0
ad_document_id           0
score                    0
category_id_doc    1251708
category_id_ad      824575
dtype: int64

In [10]:
#drop lines with nulls, ad_id and doc_id columns
document_on_ad_document_ctr_categories = document_on_ad_document_ctr_categories.dropna()
document_on_ad_document_ctr_categories.drop(document_on_ad_document_ctr_categories.columns[[0,1]],axis = 1,inplace=True)
document_on_ad_document_ctr_categories.reset_index(drop=True, inplace=True)

In [11]:
print document_on_ad_document_ctr_categories.shape
print len(document_on_ad_document_ctr_categories.category_id_doc.unique())
print len(document_on_ad_document_ctr_categories.category_id_ad.unique())

(5783348, 3)
89
92


In [12]:
#put lower category_id of both on left side
for i, row in enumerate(document_on_ad_document_ctr_categories.itertuples()):
    if row.category_id_doc > row.category_id_ad:
        ad = row.category_id_ad
        doc = row.category_id_doc
        document_on_ad_document_ctr_categories.set_value(i,'category_id_doc', ad)
        document_on_ad_document_ctr_categories.set_value(i,'category_id_ad', doc)
document_on_ad_document_ctr_categories.rename(columns={'category_id_doc' : 'category_id_l', 'category_id_ad' : 'category_id_r'},inplace=True)

In [13]:
#count how many times each pair shows, and reduce by number of shows
#keep top 20% score even if count is 1
high_score = document_on_ad_document_ctr_categories.score.quantile(q=0.8, interpolation='higher')
doc_ad_doc_count = document_on_ad_document_ctr_categories.groupby(['category_id_l','category_id_r']).score.agg({'mean_score' : 'mean', 'count' : 'count'}).reset_index()
doc_ad_doc_count = doc_ad_doc_count[(doc_ad_doc_count['count'] > 1) | (doc_ad_doc_count['mean_score'] > high_score)]
print doc_ad_doc_count.head()
print len(doc_ad_doc_count.category_id_l.unique())
print len(doc_ad_doc_count.category_id_r.unique())
print doc_ad_doc_count.mean_score.mean()
print doc_ad_doc_count.mean_score.max()
print doc_ad_doc_count.shape

   category_id_l  category_id_r  count  mean_score
0         1000.0         1000.0    557    1.467992
1         1000.0         1100.0    304    1.479777
2         1000.0         1202.0     13    1.504668
3         1000.0         1203.0     29    1.538833
4         1000.0         1204.0     29    1.469953
92
92
1.45648935831
2.75464024239
(3955, 4)


In [14]:
#create a dictionary, for each tuple (x_category,y_category) where x_category < y_category return the mean_score
dict_category = {}
for row in doc_ad_doc_count.itertuples():
    key = (row.category_id_l,row.category_id_r)
    key_r = (row.category_id_r,row.category_id_l)
    score = row.mean_score
    dict_category[key] = score
    dict_category[key_r] = score

In [15]:
#save dictionary to file
with open(path + 'dicts/dict_category_' + repr(confidence_cut_category) + '_' + repr(score_cut_category), 'ab') as handle:
    pickle.dump(dict_category, handle)

In [16]:
#remove category ids below a certain confidence level and merge promoted with documents_categories_reduced
documents_topics_reduced = documents_topics[documents_topics['confidence_level'] > confidence_cut_topic].drop("confidence_level", axis = 1)
print 'topics ids precent: ' + repr(float(len(documents_topics_reduced.topic_id.unique())) / len(documents_topics.topic_id.unique()))
print 'unique topics ids: ' + repr(len(documents_topics_reduced.topic_id.unique()))

topics ids precent: 0.9966666666666667
unique topics ids: 299


In [17]:
#take only highest scored ad_on_doc
document_on_ad_document_ctr_topics = document_on_ad_document_ctr[:document_on_ad_document_ctr.shape[0] / score_cut_topic]
print document_on_ad_document_ctr_topics.shape
print 'precentage taken: ' + repr(float(document_on_ad_document_ctr_topics.shape[0]) / document_on_ad_document_ctr.shape[0])
print 'minimal score taken: ' + repr(document_on_ad_document_ctr_topics.score.min())

(7719955, 3)
precentage taken: 0.3333333333333333
minimal score taken: 1.0352447964700002


In [18]:
print documents_topics_reduced.head()
print document_on_ad_document_ctr_topics.head()

    document_id  topic_id
10      1617787       113
16      1615583        89
25      1614235       108
32      1488264       113
37      1614048       260
   document_id  ad_document_id     score
0      1026091          898071  7.518608
1       961361         1290377  7.247171
2       961313         1585630  7.202495
3      2115556         1418830  7.192435
4      1976713          973567  7.044363


In [19]:
#merge with category ids of doc and ad
document_on_ad_document_ctr_topics = document_on_ad_document_ctr_topics.merge(documents_topics_reduced, how='left', on='document_id')
documents_topics_reduced.rename(columns={'document_id': 'ad_document_id'}, inplace=True)
document_on_ad_document_ctr_topics = document_on_ad_document_ctr_topics.merge(documents_topics_reduced, how='left', on='ad_document_id', suffixes=('_doc','_ad'))

In [20]:
print document_on_ad_document_ctr_topics.isnull().sum()
print document_on_ad_document_ctr_topics.shape

document_id             0
ad_document_id          0
score                   0
topic_id_doc      5823685
topic_id_ad       4166173
dtype: int64
(7719955, 5)


In [21]:
#drop lines with nulls, ad_id and doc_id columns
document_on_ad_document_ctr_topics = document_on_ad_document_ctr_topics.dropna()
document_on_ad_document_ctr_topics.drop(document_on_ad_document_ctr_topics.columns[[0,1]],axis = 1,inplace=True)
document_on_ad_document_ctr_topics.reset_index(drop=True, inplace=True)

In [22]:
print document_on_ad_document_ctr_topics.shape
print len(document_on_ad_document_ctr_topics.topic_id_doc.unique())
print len(document_on_ad_document_ctr_topics.topic_id_ad.unique())

(883558, 3)
299
295


In [23]:
#put lower category_id of both on left side
for i, row in enumerate(document_on_ad_document_ctr_topics.itertuples()):
    if row.topic_id_doc > row.topic_id_ad:
        ad = row.topic_id_ad
        doc = row.topic_id_doc
        document_on_ad_document_ctr_topics.set_value(i,'topic_id_doc', ad)
        document_on_ad_document_ctr_topics.set_value(i,'topic_id_ad', doc)
document_on_ad_document_ctr_topics.rename(columns={'topic_id_doc' : 'topic_id_l', 'topic_id_ad' : 'topic_id_r'},inplace=True)

In [24]:
#count how many times each pair shows, and reduce by number of shows
#keep top 20% score even if count is 1
high_score = document_on_ad_document_ctr_topics.score.quantile(q=0.8, interpolation='higher')
doc_ad_doc_count = document_on_ad_document_ctr_topics.groupby(['topic_id_l','topic_id_r']).score.agg({'mean_score' : 'mean', 'count' : 'count'}).reset_index()
doc_ad_doc_count = doc_ad_doc_count[(doc_ad_doc_count['count'] > 1) | (doc_ad_doc_count['mean_score'] > high_score)]
print doc_ad_doc_count.head()
print len(doc_ad_doc_count.topic_id_l.unique())
print len(doc_ad_doc_count.topic_id_r.unique())
print doc_ad_doc_count.mean_score.mean()
print doc_ad_doc_count.mean_score.max()
print doc_ad_doc_count.shape

   topic_id_l  topic_id_r  count  mean_score
0         0.0         0.0      5    1.472634
1         0.0         1.0      9    1.452768
2         0.0         2.0      9    1.528525
3         0.0         3.0      2    1.470104
4         0.0         4.0      2    1.569843
298
298
1.46419991871
3.79956115624
(31704, 4)


In [25]:
#create a dictionary, for each tuple (x_category,y_category) where x_category < y_category return the mean_score
dict_topic = {}
for row in doc_ad_doc_count.itertuples():
    key = (row.topic_id_l,row.topic_id_r)
    key_r = (row.topic_id_r,row.topic_id_l)
    score = row.mean_score
    dict_topic[key] = score
    dict_topic[key_r] = score

In [26]:
#save dictionary to file
with open(path + 'dicts/dict_topic_' + repr(confidence_cut_topic) + '_' + repr(score_cut_topic), 'ab') as handle:
    pickle.dump(dict_topic, handle)